# Python (EPAM, 2020), lecture 11

# Section 0. Metaclasses one more time

```python
class DisallowPublicClassAttributes(type):  # is a metaclass
    def __new__(cls, name, bases, dct):
        cls_instance = super().__new__(cls, name, bases, dct)
        if any([not key.startswith("_") for key in dct.keys()]):
            raise Exception()
        return cls_instance

class NoErrorClass(metaclass=DisallowPublicClassAttributes):
    __private = ""

class ErrorClass(metaclass=DisallowPublicClassAttributes):
    public = ""
```

# Section 1. OOP Patterns

  Three main categories:

    - Creational
        These patterns provide various object creation mechanisms,
        which increase flexibility and reuse of existing code.
    - Structural
        These patterns explain how to assemble objects and classes
        into larger structures while keeping these structures flexible and efficient.
    - Behavioral
        These patterns are concerned with algorithms and the assignment of
        responsibilities between objects.


## Creational patterns

    - Singleton
        Ensures that a class has just a single instance
    - Factory method
        Provides an interface for creating objects in
        a superclass, but allows subclasses to alter the
        type and behaviour of objects that will be created.

and others https://refactoring.guru/design-patterns/creational-patterns

    

### 3 most popular ways to implement singleton
Decorator

```python
def singleton(cls):
    instances = {}

    def getinstance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return getinstance

@singleton
class SingletonClass:
    pass
```

A base class

```python
class Singleton(object):
    _instance = None

    def __new__(cls, *args, **kwargs):
        if not isinstance(cls._instance, type(cls)):
            cls._instance = super().__new__(cls, *args, **kwargs)
        return cls._instance

class SingletonClass(Singleton):
    pass
```

A metaclass

```python
class Singleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]


class SingletonClass(metaclass=Singleton):
    pass
```

### Factory method simple example


```python
import abc
class Shape(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def calculate_area(self):
        pass

    @abc.abstractmethod
    def calculate_perimeter(self):
        pass
class Rectangle(Shape):
    def __init__(self, height, width):
        self.height = height
        self.width = width

    def calculate_area(self):
        return self.height * self.width

    def calculate_perimeter(self):
        return 2 * (self.height + self.width)

class Square(Shape):
    def __init__(self, width):
        self.width = width

    def calculate_area(self):
        return self.width ** 2

    def calculate_perimeter(self):
        return 4 * self.width
```

## Structural patterns

    - Decorator
        Lets you attach new behaviors to objects by placing these
        objects inside special wrapper objects that contain the behaviors.
    - Proxy
        Lets you provide a substitute or placeholder for another object.
        A proxy controls access to the original object, allowing you to perform
        something either before or after the request gets through to the
        original object.

and others https://refactoring.guru/design-patterns/structural-patterns

### Proxy pattern simple example

```python
class Product:
    def request(self) -> None:
        print("RealSubject: Handling request.")

class Proxy:
    def __init__(self, real_product: Product) -> None:
        self._real_product = real_product

    def request(self) -> None:
        if self.check_access():
            self._real_product.request()
            self.log_access()
        else:
            print("Forbidden")

    def check_access(self) -> bool:
        print("Proxy: Checking access prior to firing a real request.")
        return True

    def log_access(self) -> None:
        print("Proxy: Logging the time of request.", end="")
```

Python example:
    https://docs.python.org/3/library/types.html#types.MappingProxyType

## Behavioral patterns

    - Iterator
        Lets you traverse elements of a collection.
    - Observer
        Lets you define a subscription mechanism to notify multiple
        objects about any events that happen to the object they’re observing.

and others https://refactoring.guru/design-patterns/behavioral-patterns

### Observer simple example

```python
class Product:
    _observers = []
    def attach(self, observer):
        self._observers.append(observer)

    def notify(self) -> None:
        for observer in self._observers:
            observer.update(self)

    def do_some_logic(self) -> None:
        self.value = 2
        self.notify()


class ObserverA:
    def update(self, product):
        if product.value < 3:
            print("ObserverA: Reacted to the event")

product = Product()
observer_a = ObserverA()
product.attach(observer_a)
product.do_some_logic()
```

# Section 2. Garbage collection

There are two aspects to memory management and garbage collection in CPython:

    - Reference counting
    - Generational garbage collection



## Reference counting

The main garbage collection mechanism in CPython is through reference counts.
Whenever you create an object in Python, the underlying C object has both a Python type
(such as list, dict, or function) and a reference count.

At a very basic level, a Python object’s reference count is incremented whenever
the object is referenced, and it’s decremented when an object is dereferenced. If an
object’s reference count is 0, the memory for the object is deallocated.

Your program’s code can’t disable Python’s reference counting.

```python
import sys
a = 'test stirng'

assert sys.getrefcount(a) == 2
```

```python
import sys
a = 'test string'
b = [a] # Make a list with a as an element.
c = {'key': a} # Create a dictionary with a as one of the values.

assert sys.getrefcount(a) == 4
```

## Generational garbage collection

```python
class MyClass:
    pass

a = MyClass()  # refcount: 1
a.obj = a  # refcount: 2
del a  # refcount: 1
```

That's why we need a generational garbage collector.

### Generation

The garbage collector is keeping track of all objects in memory. A new object
starts its life in the first generation of the garbage collector. If Python executes
a garbage collection process on a generation and an object survives, it moves up into
a second, older generation. The Python garbage collector has three generations in
total, and an object moves into an older generation whenever it survives a garbage
collection process on its current generation.

### Threshold

For each generation, the garbage collector module has a threshold number of objects.
If the number of objects exceeds that threshold, the garbage collector will trigger
a collection process. For any objects that survive that process, they’re moved into an
older generation.

Unlike the reference counting mechanism, you may change the behavior of the
generational garbage collector in your Python program. This includes changing the
thresholds for triggering a garbage collection process in your code, manually
triggering a garbage collection process, or disabling the garbage collection process
altogether.

### Threshold

```python
import gc
gc.get_threshold()
(700, 10, 10)
gc.set_threshold(1000, 15, 15)
gc.get_threshold()
(1000, 15, 15)
```

```python
import gc
gc.get_count()

(596, 2, 1)
```

```python
import gc

gc.get_count()
(595, 2, 1)

gc.collect()
57

gc.get_count()
(18, 0, 0)
```

# Section 3. Weak references

Unlike the references we discussed above, a weak reference is a reference that does
not protect the object from getting garbage collected.

Why?

There are two main applications of weak references:

    - implement caches for large objects (weak dictionaries)
    - reduction of Pain from circular references

To create weak references, Python has provided us with a module named weakref.
A point to keep in mind before using weakref is that some builtins such as tuple or int
does not support this. list and dict support is either but we can add support through
subclassing.

## Weakref module

`class weakref.ref(object[, callback])`
    This returns a weak reference to the object.

`weakref.proxy(object[, callback])`
    This returns a proxy to object which uses a weak reference.

`weakref.getweakrefcount(object)`
    Return the number of weak references and proxies which refer to object.

`weakref.getweakrefs(object)`
    Return a list of all weak reference and proxy objects which refer to object.

## Usage of weakref

```python
import weakref

class MyClass(list):
    pass

obj = MyClass("TEST")

normal_list = obj
print(f"This is a normal list object: {normal_list}")

weak_list = weakref.ref(obj)
weak_list_obj = weak_list()
print(f"This is a object created using weak reference: {weak_list_obj}")

proxy_list = weakref.proxy(obj)
print(f"This is a proxy object: {proxy_list}")

for objects in [normal_list, weak_list_obj, proxy_list]:
    print(f"Number of weak references: {weakref.getweakrefcount(objects)}")
```

```
This is a normal object: [‘T’, ‘E’, ‘S’, ‘T’]
This is a object created using weak reference: [‘T’, ‘E’, ‘S’, ‘T’]
This is a proxy object: [‘T’, ‘E’, ‘S’, ‘T’]
Number of weak references: 2
Number of weak references: 2
Number of weak references: 0
```

# Section 4. Lazy objects

Lazy evaluation is a programming implementation paradigm that
defers evaluating necessary operations until it’s requested to do so.

Why?

```python
class User:
    def __init__(self, username):
        self.username = username
        self.profile_data = self._get_profile_data()
        print(f"{self.__class__.__name__} instance created")

    def _get_profile_data(self):
        print("Run the expensive operation")
        fetched_data = "The mock data of a large size"
        return fetched_data

def get_followers(username):
    usernames_fetched = ["David", "Aaron", "Zack"]
    users = [User(username) for username in usernames_fetched]
    return users
...
users = get_followers("user0")
```

```
Run the expensive operation
User instance created
Run the expensive operation
User instance created
Run the expensive operation
User instance created
```

## __getattr__

```python
class User2:
    def __init__(self, username):
        self.username = username
        print(f"{self.__class__.__name__} instance created")

    def __str__(self):
        return f"user {self.username}"

    def __getattr__(self, name):
        print(f"__getattr__ called for {name}")
        if name == "profile_data":
            profile_data = self._get_profile_data()
            setattr(self, name, profile_data)
            return profile_data
        else:
            raise AttributeError(f"{self} has no attribute called {name}.")

    def _get_profile_data(self):
        print("Run the expensive operation")
        fetched_data = "The mock data of a large size"
        return fetched_data
```

`__getattr__` method doesn’t get called when a particular attribute is in
the instance dictionary.